In [ ]:
def process_functionalities(clf, type_of_data, num_of_splits, bifurcation, metric=None):
    '''
    Input arguments:
        clf = type of classifier
        type_of_data = {'PEA', 'synthetic'}
        num_of_splits = number of splits for cv
        bifurcation = {'weighted','classwise'}
        metric = {'draw_roc_auc'}
    Return: X and y
    
    Info:
        PEA X and y is returned as a dataframe
        Synthetic X and y is returned as arrays
    ********************************************************
    '''
    
    list_values = list(np.round(np.arange(0, 1, 0.2),2))
    
    if type_of_data=='PEA' and  metric==None:
        X, y = get_PEA_data()
        print('PEA')
        
    elif (type_of_data=='PEA')and(metric=='draw_roc_auc'):
        X, y = get_PEA_data()
        print('draw_roc_auc')
        plot_roc_auc(clf, X, y, num_of_splits)
        
    elif (type_of_data=='synthetic')and(metric=='draw_roc_auc'):
        print('draw_roc_auc not valid with synthetic data')
        
    elif type_of_data=='synthetic':
        pass
            
    #return X, y 

In [ ]:
def get_multilabel_CM(y_test, prediction):
    
    mcm = multilabel_confusion_matrix(y_test, prediction, labels=[0,1,2])
    
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]
    
    return tn, tp, fn, fp

In [ ]:
def noise_classwise(clf, metric, class_label, folds):
    
    list_of_class_sensitivity = []
    list_values = list(np.round(np.arange(0, 1, 0.2),2))
    
    for noise in list_values:
        list_temp_A = []
        list_temp_B = []
        list_temp_C = []
        X, y = helper_data_generator.make_classification(n_samples=5000,
                                       n_features=26,
                                       n_informative=21,
                                       n_redundant=5,
                                       n_repeated=0,
                                       n_classes=3,
                                       n_clusters_per_class=1,
                                       class_sep=0.9,
                                       flip_y=noise,
                                       weights=[0.65, 0.01, 0.34])
        
#         column_size = X.shape[1]
#         cols = [x + str(i) for x, i in zip(["col"]*column_size,
#                                                range(column_size))]

#         y = pd.DataFrame(y, columns=['label'])
#         y.index = np.arange(0, len(y))
#         y.index.name = 'index'

#         syn_df = pd.DataFrame(X, columns=cols)
#         syn_df.index = np.arange(0, len(syn_df))
#         syn_df.index.name = 'index'
#         syn_df = syn_df.merge(y, on='index')
        #display(syn_df)

        #skf = StratifiedKFold(n_splits=folds, shuffle=True)
        skf = StratifiedShuffleSplit(n_splits=folds, test_size=0.2)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            #display(len(np.unique(y_test)))
            n_classes = len(np.unique(y_test))
#         rkf = KFold(n_splits=folds)
#         for train, test in rkf.split(syn_df):
#             syn_df_train = syn_df.loc[train]
#             syn_df_test = syn_df.loc[test]
#             X_train = syn_df_train.loc[:, cols].reset_index(drop=True)
#             y_train = syn_df_train.loc[:, 'label'].reset_index(drop=True)
#             X_test = syn_df_test.loc[:, cols].reset_index(drop=True)
#             y_test = syn_df_test.loc[:, 'label'].reset_index(drop=True)
#             X_train, y_train = shuffle(X_train, y_train)
            #print(clf)
            clf.fit(X_train, y_train)
            prediction = clf.predict(X_test)
            
            if metric=='roc_auc_metric':
                y_test = label_binarize(y_test, classes=[0,1,2])
                prediction = label_binarize(prediction, classes=[0,1,2])
                fpr = dict()
                tpr = dict()
                roc_auc = dict()
                # https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
                for i in range(n_classes):
                    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], prediction[:, i])
                    roc_auc[i] = auc(fpr[i], tpr[i])
                #print('roc_auc_class: ',roc_auc)

                list_temp_A.append(roc_auc[0])
                list_temp_B.append(roc_auc[1])
                list_temp_C.append(roc_auc[2])
            elif metric=='precision_recall':
                y_test = label_binarize(y_test, classes=[0,1,2])
                prediction = label_binarize(prediction, classes=[0,1,2])
                precision = dict()
                recall = dict()
                pr_auc = dict()
                # https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
                for i in range(n_classes):
                    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], prediction[:, i])
                    pr_auc[i] = auc(recall[i], precision[i])
                #print('roc_auc_class: ',roc_auc)

                list_temp_A.append(pr_auc[0])
                list_temp_B.append(pr_auc[1])
                list_temp_C.append(pr_auc[2])
            else:
                # tn, tp, fn, fp = get_multilabel_CM(y_test, prediction)
                if metric=='fnr':
                    fnr_class_A, fnr_class_B, fnr_class_C = get_scores(y_test, prediction, metric)
                    list_temp_A.append(fnr_class_A)
                    list_temp_B.append(fnr_class_B)
                    list_temp_C.append(fnr_class_C)
                elif metric=='fpr':
                    fpr_class_A, fpr_class_B, fpr_class_C = get_scores(y_test, prediction, metric)
                    list_temp_A.append(fpr_class_A)
                    list_temp_B.append(fpr_class_B)
                    list_temp_C.append(fpr_class_C)
                elif metric=='sensitivity':
                    sensitivity_recall_tpr_class_A, sensitivity_recall_tpr_class_B, sensitivity_recall_tpr_class_C = get_scores(metric, y_test, prediction)
                    list_temp_A.append(sensitivity_recall_tpr_class_A)
                    list_temp_B.append(sensitivity_recall_tpr_class_B)
                    list_temp_C.append(sensitivity_recall_tpr_class_C)
                elif metric=='specificity':
                    speificity_neg_recall_tnr_class_A, speificity_neg_recall_tnr_class_B, speificity_neg_recall_tnr_class_C = get_scores(y_test, prediction, metric)
                    list_temp_A.append(speificity_neg_recall_tnr_class_A)
                    list_temp_B.append(speificity_neg_recall_tnr_class_B)
                    list_temp_C.append(speificity_neg_recall_tnr_class_C)
                
        list_of_class_sensitivity.append((np.mean(list_temp_A),np.mean(list_temp_B),np.mean(list_temp_C)))
    
    if class_label=='0':
        return list(np.round(list_of_class_sensitivity,2)[:,0]) # class A or -1
    elif class_label=='1':
        return list(np.round(list_of_class_sensitivity,2)[:,1]) # class A or -1
    elif class_label=='2':
        return list(np.round(list_of_class_sensitivity,2)[:,2]) # class A or -1
    
def create_dict_clf_noise_classwise(classifier_list, metric, class_label, folds):
    
    list_lists_clf = []
    list_lists_clf_metric = []

    for i in range(len(classifier_list)):
        list_lists_clf.append('list'+str(i))

    for i in range(len(list_lists_clf)):
        list_lists_clf[i] = []
    
    print(metric)
    for i in range(len(classifier_list)):
        if isinstance(classifier_list[i],PerformanceEnrichmentAnalysisClassifier):
            name = 'PEA'
            list_lists_clf_metric.append((name,noise_classwise(classifier_list[i], metric, class_label, folds)))
        else:
            list_lists_clf_metric.append((str(classifier_list[i]).split('(')[0],noise_classwise(classifier_list[i], metric, class_label, folds)))
            #list_lists_clf_metric.append((str(classifier_list[i]).split('(')[0],noise(classifier_list[i], metric, folds)))
    
    dict_items= {}

    for i in range(np.array(list_lists_clf_metric).shape[0]):
        dict_items[np.unique(np.array(list_lists_clf_metric).T[0][i])[0]] = np.hstack((np.array(list_lists_clf_metric).T[1][i]))

    create_dataframe_classwise(dict_items, class_label, metric)
    
def create_dataframe_classwise(dict_items, class_label, metric):
    
    list_lists_clf = []

    for i in range(5):
        list_lists_clf.append('list'+str(i))

    for i in range(5):
        list_lists_clf[i] = []

    for i in range(5):
        for key, value in dict_items.items():
            #print(key, value)
            list_lists_clf[i].append((key,value[i]))
        #print(len(value))
        #print(np.array(dict_items['SVC'].T).reshape(-1,1))

    #print(list_lists_clf)

    list_lists_clf = np.array(list_lists_clf)
    a = list_lists_clf.T[0].reshape(1,-1)
    a = a[0]

    _, idx = np.unique(a, return_index=True)
    x = a[np.sort(idx)]

    list_noise_vals = list(np.round(np.arange(0, 1, 0.2),2))
    list_noise_vals = np.array(list_noise_vals)

    y = list_lists_clf.T[1]

    col_noise = []
    for i in list_noise_vals:
        col_noise.append(('noise_'+str(i)))
        
    plot_bar_classwise(x, y, col_noise, metric, class_label)
    
def plot_bar_classwise(x, y, col_noise, metric, class_label):
    # https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
    # https://blog.finxter.com/matplotlib-legend/

    df_1 = pd.DataFrame(y.T, index=col_noise, columns=x, dtype='float')
    display(df_1)
    ax = df_1.plot(kind='bar',figsize=(16, 7.5),width=0.8, cmap='jet', alpha=0.8)
    ax.set_yticks(np.arange(0,1.1,0.1))
    ax.minorticks_on()
    ax.set_ylim(0,2)
    ax.grid(which='major', linestyle='-', linewidth='0.3', color='green', alpha=0.7)
    #ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    ax.set_xticklabels(df_1.index,fontsize=18,rotation=0)
    ax.set_facecolor('white')
    ax.set_ylabel(str(metric),fontsize=18)
    ax.set_title('Noise vs '+str(metric)+'_class_'+class_label,fontsize=18)

    # facecolor='grey',
    ax.legend(['PEA: Mean = '+ str(round(df_1.mean(),2)[0]) +'& SD = '+str(round(df_1.std(),2)[0]),
               'KNeighborsClassifier: Mean = '+ str(round(df_1.mean(),2)[1]) +'& SD = '+str(round(df_1.std(),2)[1]),
               'SVC: Mean = '+ str(round(df_1.mean(),2)[2]) +'& SD = '+str(round(df_1.std(),2)[2]),
               'DecisionTreeClassifier: Mean = '+ str(round(df_1.mean(),2)[3]) +'& SD = '+str(round(df_1.std(),2)[3]),
               'AdaBoostClassifier: Mean = '+ str(round(df_1.mean(),2)[4]) +'& SD = '+str(round(df_1.std(),2)[4])],
               fontsize=12, framealpha=0.3, facecolor='grey', edgecolor='black', title= str(metric)+'_class_'+class_label+' - Central Tendency', title_fontsize=14, borderpad=.9)

    # legend = ax.legend()
    # frame = legend.get_frame()
    # frame.set_color('grey')

    labels = list(round(df_1.std(),2))
    add_value_labels(ax)

In [ ]:
def get_scores(y_test, prediction, metric):
    
#     list_temp_A = []
#     list_temp_B = []
#     list_temp_C = []
    
    tn, tp, fn, fp = get_multilabel_CM(y_test, prediction)
    
    score_class_A = []
    score_class_B = []
    score_class_C = []
    
    if metric=='fnr':
        score_class_A = (fn / (fn + tp))[0]
        score_class_B = (fn / (fn + tp))[1]
        score_class_C = (fn / (fn + tp))[2]
    elif metric=='fpr':
        score_class_A = (fp / (tn + fp))[0]
        score_class_B = (fp / (tn + fp))[1]
        score_class_C = (fp / (tn + fp))[2]
    elif metric=='sensitivity':
        score_class_A = (tp / (tp + fn))[0]
        score_class_B = (tp / (tp + fn))[1]
        score_class_C = (tp / (tp + fn))[2]
    elif metric=='specificity':
        score_class_A = (tn / (tn + fp))[0]
        score_class_B = (tn / (tn + fp))[1]
        score_class_C = (tn / (tn + fp))[2]
        
    return score_class_A, score_class_B, score_class_C

def process_function():
    pass